<a href="https://colab.research.google.com/github/JiihwanKim/Quantum_Information_Processing/blob/master/Guessing_for_Randomness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Randomness in Measurement Device Independent QRNG

 Randomness in Measurement Device Independent QRNG
 
 This function has four required arguments:
 
    p: prior probability list (1xN)
    
    RHO: density operators(dxd array) list (1xN)
    
    Pobs: probability distribution list to be observed (NxO)
    
    η: detection accuracy (1x1)
    
    where N is the number of states and O is the number of outcomes.
    
 Pg,M = Guessing_for_Randomness(p,RHO,Pobs,η): calculates guessing
 
 probability(Pg), optimal measurement(M).
 
 Requires: cvxpy, numpy
 
 Author: Jihwan Kim (mailto://mslab.k@gmail.com )
 
 Affiliation: Mathematical Science Lab, Applied Physics, Hanyang University, South Korea
 
 Reference:  https://arxiv.org/abs/1703.03330v1
 
 Last updated: Jun. 21, 2019

## Preparation

In [0]:
# Package Settings
import cvxpy as cp
import numpy as np

# Definition of Function
def Guessing_for_Randomness(p,RHO,Pobs,η):
  def Correct_Probability(p,RHO,M):
    return cp.sum([cp.trace(p[a]*RHO[a]*cp.sum([M[a][x][x] for x in range(num_of_outcomes)])) for a in range(num_of_states)])
  num_of_states = len(RHO)
  num_of_outcomes = len(Pobs[0])
  # Construction of the problem.
  M = [[[cp.Variable((len(RHO[0]),len(RHO[0])),symmetric=True) for x in range(num_of_outcomes)] for x in range(num_of_outcomes)] for x in range(num_of_states)];
  objective = cp.Maximize(Correct_Probability(p,RHO,M));
  constraints = [M[a][x][e]>>0 for x in range(num_of_outcomes) for e in range(num_of_outcomes) for a in range(num_of_states)];
  constraints += [cp.sum([M[a][x][e] for x in range(num_of_outcomes) for e in range(num_of_outcomes)])==np.eye(len(RHO[1])) for a in range(num_of_states)]
  constraints += [cp.sum([M[a][x][e] for x in range(num_of_outcomes)])==cp.sum([M[a][x][e] for x in range(num_of_outcomes)])[0][0]*np.eye(len(RHO[1])) for e in range(num_of_outcomes) for a in range(num_of_states)]
  constraints += [cp.sum([M[a][x][e] for e in range(num_of_outcomes)])==cp.sum([M[1][x][e] for e in range(num_of_outcomes)]) for x in range(num_of_outcomes) for a in range(num_of_states)]
  constraints += [cp.sum( [p[a]*cp.trace(M[a][x][e]*RHO[a]) for e in range(num_of_outcomes)] ) == Pobs[a][x] for x in range(num_of_outcomes) for a in range(len(Pobs))]
  prob = cp.Problem(objective, constraints);
  prob.solve()

  # Return the optimals.
  Pg = objective.value
  M = [[[np.around(M[a][x][e].value) for e in range(num_of_outcomes)] for x in range(num_of_outcomes)] for a in range(num_of_states)]
  return Pg,M

## Example

In [2]:

# Prior Probability of Quantum States
p = [0.99, 0.01] 

# Quantum states
RHO = [np.array([[.5,.5],[.5,.5]])]
RHO.append(np.array([[1,0],[0,0]]))

# Detection Quality
η= 0.9999999

# Probability Distribution for Users
Pobs = [[(η*.5+(1-η)/2)*p[0],(η*.5+(1-η)/2)*p[0]],[(η*1+(1-η)/2)*p[1],(η*0+(1-η)/2)*p[1]]]


Pg,M = Guessing_for_Randomness(p,RHO,Pobs,η)
print(Pg)

0.5054426343251976
